In [3]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from numpy.linalg import norm, solve
import numpy as np

import pinocchio

In [21]:
target_state = np.loadtxt("optimal_solution.txt", max_rows=1).T
target_position = target_state[1:8]
target_velocity = target_state[8:15]

data = np.loadtxt("optimal_solution.txt", skiprows=1).T
time = data[0]
position = data[1:8]
velocity = data[8:15]
acceleration = data[15:22]

position_limit = np.array([[-2.8973, -1.7628, -2.8973, -3.0718, -2.8973, -0.0175, -2.8973], 
                          [2.8973, 1.7628, 2.8973, -0.0698, 2.8973, 3.7525, 2.8973]])

velocity_limit = np.array([2.1750, 2.1750, 2.1750, 2.1750, 2.6100, 2.6100, 2.6100])

acceleration_limit = np.array([15.0, 7.5, 10.0, 12.5, 15.0, 20.0, 20.0])


fig = make_subplots(rows=7, cols=3)

for i in range(7):

     # Plot position trajectory, limit and target
    fig.add_trace(
    go.Scatter(x=time, y=position[i, :]),
    row=i+1, col=1 )

    fig.add_hline(y=target_position[i],
    row=i+1, col=1, line_dash="dash", line_width=0.5)

    fig.add_hline(y=position_limit[0, i],
    row=i+1, col=1, line_dash="dash", line_width=1)

    fig.add_hline(y=position_limit[1, i],
    row=i+1, col=1, line_dash="dash", line_width=1)

    # Plot velocitiy trajectory, limit and target
    fig.add_trace(
    go.Scatter(x=time, y=velocity[i, :]),
    row=i+1, col=2 )

    fig.add_hline(y=target_velocity[i],
    row=i+1, col=2, line_dash="dash", line_width=0.5)

    fig.add_hline(y=velocity_limit[i],
    row=i+1, col=2, line_dash="dash", line_width=1)

    fig.add_hline(y=-velocity_limit[i],
    row=i+1, col=2, line_dash="dash", line_width=1)

    # Plot acceleration trajectory and limit
    fig.add_trace(
    go.Scatter(x=time, y=acceleration[i, :]),
    row=i+1, col=3 )

    fig.add_hline(y=acceleration_limit[i],
    row=i+1, col=3, line_dash="dash", line_width=1)

    fig.add_hline(y=-acceleration_limit[i],
    row=i+1, col=3, line_dash="dash", line_width=1)
    
fig.update_layout(height=2000, width=1000, title_text="PolyMPC collocation_fix_jw branch, poly=10, segment=1")
fig.update(layout_showlegend=False)
fig.show()